In [20]:
import pandas as pd
from sklearn import preprocessing

# Dimensão Uso

dfUso = pd.read_csv('../Dados/Processed/Uso/Consolidado.csv')

# Normalizar

dfUso = pd.read_csv('../Dados/Processed/Uso/Consolidado.csv')

dfUsoID     = dfUso[['Date', 'Região']]
dfUsoValues = dfUso[['Bancarização', 'Nível Crédito', 'Nível Depósito']]

scaler = preprocessing.MinMaxScaler().fit(dfUsoValues.values)
oemNormalized = scaler.transform(dfUsoValues.values)
oemNormalized = pd.DataFrame(oemNormalized)

oemNormalized = oemNormalized.rename(columns = {0:'Bancarização',
                                                1:'Nível de Crédito',
                                                2:'Nível de Depósito'})

dfUso = pd.concat([dfUsoID, oemNormalized], axis = 1).dropna()

# Dimensão Acesso

dfGeog = pd.read_csv('../Dados/Processed/Acesso/Geofrafico.csv')
dfDemo = pd.read_csv('../Dados/Processed/Acesso/Demografico.csv')

dfGeog = dfGeog[(dfGeog.Date >= 2014) & (dfGeog.Date <= 2019)].reset_index(drop = True)
dfDemo = dfDemo[(dfDemo.Date >= 2014) & (dfDemo.Date <= 2019)].reset_index(drop = True)

dfGeog = dfGeog.fillna(0)
dfDemo = dfDemo.fillna(0)

dfGeog

dfDemo

# Normalizar

dfGeogID     = dfGeog[dfGeog.columns[0:2]]
dfGeogValues = dfGeog[dfGeog.columns[2:]]

scaler = preprocessing.MinMaxScaler().fit(dfGeogValues.values)
dfGeogNormalized = scaler.transform(dfGeogValues.values)
dfGeogNormalized = pd.DataFrame(dfGeogNormalized, columns = dfGeogValues.columns)

dfGeog = pd.concat([dfGeogID, dfGeogNormalized], axis = 1)

# Normalizar

dfDemoID     = dfDemo[dfDemo.columns[0:2]]
dfDemoValues = dfDemo[dfDemo.columns[2:]]

scaler = preprocessing.MinMaxScaler().fit(dfDemoValues.values)
dfDemoNormalized = scaler.transform(dfDemoValues.values)
dfDemoNormalized = pd.DataFrame(dfDemoNormalized, columns = dfDemoValues.columns)

dfDemo = pd.concat([dfDemoID, dfDemoNormalized], axis = 1)

dfInclusao = dfDemo.merge(dfGeog).merge(dfUso)

## Cálculo Índice

columns = ['Total invesment activity - Celular',
       'Number Fintechs Brazil - Celular', 'Number Fintechs Acumulado_Geo', 'Total invesment activity_Geo', 'Number Fintechs Brazil_Geo', 'Total invesment activity_Demo', 'Number Fintechs Brazil_Demo', 'Number Fintechs Brazil - Celular_Geo', 'Total invesment activity - Celular_Geo', 'Total invesment activity - Celular_Demo', 'Number Fintechs Acumulado_Demo', 'Number Fintechs Brazil - Celular_Demo']

dfInclusaoFintechs = dfInclusao[['Date', 'Região'] + columns]
dfInclusao = dfInclusao.drop(columns = columns)

denominador = (dfInclusao.shape[1] - 2) ** 0.5

dfInclusao['Índice de Inclusão Financeira'] = 0

for c in dfInclusao.columns[2:-1]:
    distanciaLocal = (1 - dfInclusao[c]) ** 2
    dfInclusao['Índice de Inclusão Financeira'] = dfInclusao['Índice de Inclusão Financeira'] + distanciaLocal

dfInclusao['Índice de Inclusão Financeira'] = 1 - dfInclusao['Índice de Inclusão Financeira'] ** 0.5 / denominador

dfInclusao = dfInclusao.merge(dfInclusaoFintechs)

dfInclusao.columns

columns = ['Date', 'Região', 'Índice de Inclusão Financeira',
       'Number of headquarters of authorized financial institutions_Demo',
       'Number of branches of financial institutions_Demo',
       'Number of banking service outposts of financial institutions_Demo',
       'Number of electronic service outposts of financial institutions_Demo',
       'Number of correspondents of financial institutions _Demo',
       'Number of ATMs of financial institutions_Demo',
       'Number of POSs of financial institutions_Demo',
       'Number of authorized credit unions _Demo',
       'Number of credit unions banking service outposts_Demo',
       'Number of credit unions electronic service outposts_Demo',
       'Number of credit unions correspondents_Demo',
       'Number of branches of finance companies_Demo',
       'Number of banking services outpost of finance companies_Demo',
       'Number of electronic service outposts of finance companies_Demo',
       'Number of microentrepreneur, and small business credit companies correspondents_Demo',
       'Total invesment activity', 'Number Fintechs Brazil',
       'Number of headquarters of authorized financial institutions_Geo',
       'Number of branches of financial institutions_Geo',
       'Number of banking service outposts of financial institutions_Geo',
       'Number of electronic service outposts of financial institutions_Geo',
       'Number of correspondents of financial institutions _Geo',
       'Number of ATMs of financial institutions_Geo',
       'Number of POSs of financial institutions_Geo',
       'Number of authorized credit unions _Geo',
       'Number of credit unions banking service outposts_Geo',
       'Number of credit unions electronic service outposts_Geo',
       'Number of credit unions correspondents_Geo',
       'Number of branches of finance companies_Geo',
       'Number of banking services outpost of finance companies_Geo',
       'Number of electronic service outposts of finance companies_Geo',
       'Number of microentrepreneur, and small business credit companies correspondents_Geo',
       'Bancarização', 'Nível de Crédito', 'Nível de Depósito',
       'Total invesment activity - Celular',
       'Number Fintechs Brazil - Celular', 'Number Fintechs Acumulado_Geo',
       'Total invesment activity_Geo', 'Number Fintechs Brazil_Geo',
       'Total invesment activity_Demo', 'Number Fintechs Brazil_Demo',
       'Number Fintechs Brazil - Celular_Geo',
       'Total invesment activity - Celular_Geo',
       'Total invesment activity - Celular_Demo',
       'Number Fintechs Acumulado_Demo',
       'Number Fintechs Brazil - Celular_Demo']

dfInclusao[columns].to_csv('../Dados/Processed/Índice de Inclusão Financeira.csv', index = None)

